In [ ]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle

# Make sdf from parcel master

In [ ]:

filePath = "F:/GIS/PARCELUPDATE/Workspace/"
sdeEdit    = os.path.join(filePath, "Edit.sde")
# parcel feature class years to use for all cumulative accounting data
years = [2012, 2018, 2019, 2020, 2021, 2022, 2023]

# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
# filter to years
sdf_units = sdf_units.loc[sdf_units['YEAR'].isin(years)]

In [ ]:
# Bring in demographic data for households and population
sr = arcpy.SpatialReference(26910)
# block group feature layer polygons
block_groups_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'
sdf_census_geo = get_fs_data_spatial(block_groups_url)
sdf_block = sdf_census_geo.loc[(sdf_census_geo['YEAR'] == 2020) & (sdf_census_geo['GEOGRAPHY'] == 'Block Group')]
sdf_block.spatial.sr = sr
sdf_tract = sdf_census_geo.loc[(sdf_census_geo['YEAR'] == 2020) & (sdf_census_geo['GEOGRAPHY'] == 'Tract')]
sdf_tract.spatial.sr = sr

In [ ]:
# bring in demogrpahic data
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'
df_census = get_fs_data(census_url)

# Generate a list of variables and geographies that we want to summarize at the parcel level

In [ ]:
import numpy as np

def calculate_occupancy_rates(df, year, occupancy_vars):
    """
    Calculate occupancy and seasonal rates at the block group level.

    Parameters:
    df (pd.DataFrame): Census data with 'TRPAID', 'variable_code', 'value', and 'Year' columns.
    year (int): The year of interest.
    occupancy_vars (dict): A dictionary mapping descriptive variable names to census codes.
                           Example: {
                               'vacant_units': 'B25002_003E',
                               'occupied_units': 'B25002_002E',
                               'seasonal_units': 'B25004_006E'
                           }

    Returns:
    pd.DataFrame: DataFrame with occupancy and seasonal rates.
    """
    
    # Filter data for selected year and census codes
    df_filtered = df.query("variable_code in @occupancy_vars.values() and year_sample == @year")
    
    # Pivot to wide format
    df_pivot = df_filtered.pivot(index='TRPAID', columns='variable_code', values='value')
    
    # Rename columns based on the provided dictionary
    df_pivot = df_pivot.rename(columns={v: k for k, v in occupancy_vars.items()})

    # Calculate total units
    df_pivot['total_units'] = df_pivot['vacant_units'] + df_pivot['occupied_units']

    # Avoid division by zero
    df_pivot['occupancy_rate'] = df_pivot['occupied_units'] / df_pivot['total_units'].replace(0, np.nan)
    df_pivot['seasonal_rate'] = df_pivot['seasonal_units'] / df_pivot['total_units'].replace(0, np.nan)

    # Reset index for clarity
    return df_pivot.reset_index()


In [ ]:
occupancy_codes = {
    'vacant_units': 'B25002_003E',
    'occupied_units': 'B25002_002E',
    'seasonal_units': 'B25004_006E',
    'household_size': 'B25010_001E'
}
# We should add population
df_census = df_census.loc[df_census['sample_level'] == 'block group']
df_occupancy = calculate_occupancy_rates(df_census, 2023, occupancy_codes)

In [ ]:
with arcpy.EnvManager(outputZFlag="Disabled"):
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features=r"F:\GIS\DB_CONNECT\Vector.sde\SDE.Census\SDE.Tahoe_Census_Geography",
        Output_Geodatabase=r"C:\GIS\Scratch.gdb"
    )

sdf_block_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/Tahoe_Census_Geography')
sdf_block_noz = sdf_block_noz.loc[(sdf_block_noz['YEAR'] == 2020) & (sdf_block_noz['GEOGRAPHY'] == 'Block Group')]
sdf_block_noz.spatial.sr = sr

sdf_tract_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/Tahoe_Census_Geography')
sdf_tract_noz = sdf_tract_noz.loc[(sdf_tract_noz['YEAR'] == 2020) & (sdf_tract_noz['GEOGRAPHY'] == 'Tract')]    
sdf_tract_noz.spatial.sr = sr

In [ ]:
sdf_tract_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/Tahoe_Census_Geography')
sdf_tract_noz = sdf_tract_noz.loc[(sdf_tract_noz['YEAR'] == 2020) & (sdf_tract_noz['GEOGRAPHY'] == 'Tract')]    
sdf_tract_noz.spatial.sr = sr

In [ ]:
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'

In [ ]:
# spatial join to get Block Group
arcpy.SpatialJoin_analysis(sdf_units, sdf_block_noz, "Existing_Development_BlockGroup", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Tract
arcpy.SpatialJoin_analysis(sdf_units, sdf_tract_noz, "Existing_Development_Tract", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")

In [ ]:

sdf_units_block = pd.DataFrame.spatial.from_featureclass("Existing_Development_BlockGroup", sr=sr)
sdf_units_tract = pd.DataFrame.spatial.from_featureclass("Existing_Development_Tract", sr=sr)

In [ ]:
sdfParcel = sdf_units

In [ ]:
sdfParcel['BLOCK_GROUP']   = sdfParcel.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
sdfParcel['TRACT']        = sdfParcel.APN.map(dict(zip(sdf_units_tract.APN, sdf_units_tract.TRPAID)))

In [ ]:
sdfParcel.to_pickle('parcel_pickle_part1.pkl')

In [ ]:
sdfParcel_2023 = sdfParcel.loc[sdfParcel['YEAR'] == 2023]

In [ ]:
sdfParcel_2023['occupied_rate'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_occupancy.TRPAID, df_occupancy.occupancy_rate)))
sdfParcel_2023['seasonal_rate'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_occupancy.TRPAID, df_occupancy.seasonal_rate)))
sdfParcel_2023['household_size'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_occupancy.TRPAID, df_occupancy.household_size)))

In [ ]:
sdfParcel_2023['occupied_units']=   sdfParcel_2023['occupied_rate'] * sdfParcel_2023['Residential_Units']
sdfParcel_2023['seasonal_units']=  sdfParcel_2023['seasonal_rate'] * sdfParcel_2023['Residential_Units']
sdfParcel_2023['num_of_residents'] = sdfParcel_2023['household_size'] * sdfParcel_2023['occupied_units']

In [ ]:
tesselation = get_fs_data_spatial('https://maps.trpa.org/server/rest/services/Transportation_Equity_Analysis_Tessellation/FeatureServer/0')

In [ ]:
# spatial join to get GRID_ID
arcpy.SpatialJoin_analysis(sdf_units, tesselation, "Existing_Development_Hex", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")

In [ ]:
sdf_units_hex = pd.DataFrame.spatial.from_featureclass("Existing_Development_Hex", sr=sr)
sdfParcel['GRID_ID']        = sdfParcel.APN.map(dict(zip(sdf_units_tract.APN, sdf_units_tract.TRPAID)))

In [ ]:
sdfParcel_2023['GRID_ID'] = sdfParcel_2023.APN.map(dict(zip(sdf_units_hex.APN, sdf_units_hex.GRID_ID)))

In [ ]:
# group by GRID_ID and sum the values for occupied_units, seasonal_units, and num_of_residents
sdfParcel_2023_grouped = sdfParcel_2023.groupby('GRID_ID').agg({
    'occupied_units': 'sum',
    'seasonal_units': 'sum',
    'num_of_residents': 'sum'
}).reset_index()
sdfParcel_2023_grouped['total_units'] = sdfParcel_2023_grouped['occupied_units'] + sdfParcel_2023_grouped['seasonal_units']
sdfParcel_2023_grouped['occupied_unit_rate'] = sdfParcel_2023_grouped['occupied_units']/sdfParcel_2023_grouped['total_units'].replace(0, np.nan)
sdfParcel_2023_grouped['seasonal_unit_rate'] = sdfParcel_2023_grouped['seasonal_units']/sdfParcel_2023_grouped['total_units'].replace(0, np.nan)

In [ ]:
sdfParcel_2023_grouped['occupied_unit_rate'] = sdfParcel_2023_grouped['occupied_unit_rate'].fillna(0)
sdfParcel_2023_grouped['seasonal_unit_rate'] = sdfParcel_2023_grouped['seasonal_unit_rate'].fillna(0)

In [ ]:
tesselation_attributed = tesselation.merge(sdfParcel_2023_grouped, on='GRID_ID', how='left')
tesselation_attributed['occupied_unit_rate'] = tesselation_attributed['occupied_unit_rate'].fillna(0)
tesselation_attributed['seasonal_unit_rate'] = tesselation_attributed['seasonal_unit_rate'].fillna(0)
tesselation_attributed['occupied_units'] = tesselation_attributed['occupied_units'].fillna(0)
tesselation_attributed['seasonal_units'] = tesselation_attributed['seasonal_units'].fillna(0)
tesselation_attributed['num_of_residents'] = tesselation_attributed['num_of_residents'].fillna(0)
tesselation_attributed['total_units'] = tesselation_attributed['total_units'].fillna(0)

In [ ]:
# export tesselation_attributed to a shapefile correctly
output_path = 'C:\GIS\Scratch.gdb\Tessellation_Attributed'   
arcpy.management.CopyFeatures(tesselation_attributed, output_path)

In [ ]:
# make a feature class from the centroid of tesselation attributed
export_fc = 'C:\GIS\Scratch.gdb\Tessellation_2023'
arcpy.management.FeatureToPoint(tesselation_attributed, export_fc, "CENTROID")